In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#-----
from DOVSAudit import DOVSAudit
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer

In [ ]:
save_dfs=True

date_0 = '2024-02-04'
date_1 = '2024-02-10'
states=None
# opcos='oh'
opcos=None
CI_NB_min=15
# CI_NB_min=None
conn_outages = Utilities.get_utldb01p_oracle_connection()

In [ ]:
#--------------------------------------------------
sql_outage_full = DOVSOutages_SQL.build_sql_std_outage(
    mjr_mnr_cause=None, 
    include_premise=True, 
    date_range=[date_0, date_1], 
    states=states, 
    opcos=opcos, 
    CI_NB_min=CI_NB_min
).get_sql_statement()
#-----
print(f'sql_outage_full:\n{sql_outage_full}\n\n')
#-----
df_outage_OG = pd.read_sql_query(
    sql_outage_full, 
    conn_outages, 
    dtype={
        'CI_NB':np.int32, 
        'CMI_NB':np.float64, 
        'OUTG_REC_NB':np.int32
    }
)
#-----
print(f"df_outage_OG.shape = {df_outage_OG.shape}")
print(f"# OUTG_REC_NBs     = {df_outage_OG['OUTG_REC_NB'].nunique()}")

In [ ]:
#--------------------------------------------------
sql_outage_i = DOVSOutages_SQL.build_sql_std_outage(
    mjr_mnr_cause=None, 
    include_premise=True, 
    premise_nbs = [df_outage_OG['PREMISE_NB'].unique()[0]], 
    states=states, 
    opcos=opcos, 
    CI_NB_min=CI_NB_min
).get_sql_statement()
#-----
print(f'sql_outage_i:\n{sql_outage_i}\n\n')
#-----
df_outage_i = pd.read_sql_query(
    sql_outage_i, 
    conn_outages, 
    dtype={
        'CI_NB':np.int32, 
        'CMI_NB':np.float64, 
        'OUTG_REC_NB':np.int32
    }
)
#-----
print(f"df_outage_OG.shape = {df_outage_i.shape}")
print(f"# OUTG_REC_NBs     = {df_outage_i['OUTG_REC_NB'].nunique()}")

In [ ]:
df_outage_i

In [ ]:
#--------------------------------------------------
sql_outage_i = DOVSOutages_SQL.build_sql_std_outage(
    mjr_mnr_cause=None, 
    include_premise=True, 
    premise_nbs = df_outage_OG['PREMISE_NB'].unique().tolist(), 
    states=states, 
    opcos=opcos, 
    CI_NB_min=CI_NB_min
).get_sql_statement()
#-----
print(f'sql_outage_i:\n{sql_outage_i}\n\n')
#-----
df_outage_i = pd.read_sql_query(
    sql_outage_i, 
    conn_outages, 
    dtype={
        'CI_NB':np.int32, 
        'CMI_NB':np.float64, 
        'OUTG_REC_NB':np.int32
    }
)
#-----
print(f"df_outage_OG.shape = {df_outage_i.shape}")
print(f"# OUTG_REC_NBs     = {df_outage_i['OUTG_REC_NB'].nunique()}")

In [ ]:
dovs = DOVSOutages(
    build_sql_function = DOVSOutages_SQL.build_sql_std_outage, 
    build_sql_function_kwargs = dict(
        mjr_mnr_cause=None, 
        include_premise=True, 
        premise_nbs = df_outage_OG['PREMISE_NB'].unique().tolist(), 
        states=states, 
        opcos=opcos, 
        CI_NB_min=CI_NB_min, 
        field_to_split='premise_nbs', 
        batch_size=1000, 
        verbose=True, 
        n_update=10
    )
)

In [ ]:
dovs.df

In [ ]:
dovs_df = dovs.df

In [ ]:
dovs_df.set_index('PREMISE_NB')

In [ ]:
dovs_df.groupby('PREMISE_NB')['OUTG_REC_NB']

In [ ]:
consol_df = Utilities_df.consolidate_df(
    df = dovs_df.iloc[:1000][['PREMISE_NB', 'OUTG_REC_NB', 'DT_OFF_TS_FULL', 'DT_ON_TS']], 
    groupby_cols = ['PREMISE_NB'], 
    cols_shared_by_group = None, 
    cols_to_collect_in_lists = None, 
    as_index=True, 
    include_groupby_cols_in_output_cols=False, 
    allow_duplicates_in_lists=False, 
    allow_NaNs_in_lists=False, 
    recover_uniqueness_violators=True, 
    gpby_dropna=True, 
    rename_cols=None, 
    custom_aggs_for_list_cols=None, 
    verbose=True
)
consol_df

In [ ]:
def get_outages_timeline(
    a
):
    r"""
    Returns the outages suffered as a function of time
    """